**Top 4% achieved in Kaggle**

In [9]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [10]:
kaggle_train_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Others/Second/Kaggle/train.csv')

In [11]:
kaggle_test_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Others/Second/Kaggle/test.csv')

In [12]:
mnist_data = tf.keras.datasets.mnist

In [13]:
(train_images, train_labels), (test_images, test_labels) = mnist_data.load_data()

In [14]:
train_images = train_images/255.0
test_images = test_images/255.0
train_images = train_images.reshape(60000,28,28,1)
test_images = test_images.reshape(10000,28,28,1)

In [15]:
kaggle_train_labels = kaggle_train_data.iloc[:, 0]
kaggle_train_images = kaggle_train_data.iloc[:, 1:]

In [16]:
kaggle_train_images = kaggle_train_images/255.0
kaggle_test_data = kaggle_test_data/255.0

In [17]:
kaggle_train_images = kaggle_train_images.to_numpy()
kaggle_test_data = kaggle_test_data.to_numpy()

In [18]:
kaggle_train_images = kaggle_train_images.reshape(42000,28,28,1)
kaggle_test_data = kaggle_test_data.reshape(28000,28,28,1)

## Model

In [19]:
model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(128, (5,5), activation='relu', input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D(2,2), 
      tf.keras.layers.Conv2D(64, (5,5), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2), 
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2), 

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dropout(.2),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(.2),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dropout(.2),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dropout(.2),
      tf.keras.layers.Dense(10, activation='softmax')
])

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [57]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>=0.9989):
      print("\nReached 99.89% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [58]:
model.fit(train_images, train_labels, epochs=50, batch_size=5000, validation_data=(test_images, test_labels), callbacks=callbacks)

Epoch 1/10
12/12 [==============================] - ETA: 0s - loss: 0.0022 - accuracy: 0.9995WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0050s vs `on_test_batch_end` time: 0.0643s). Check your callbacks.

Reached 99.7% accuracy so cancelling training!
12/12 [==============================] - 2s 193ms/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.0031 - val_accuracy: 0.9991


In [53]:
outputs = model.predict(kaggle_test_data)
#outputs

In [54]:
my_predictions = []
for i in outputs:
    my_predictions.append(np.argmax(i))

In [55]:
output_file = pd.DataFrame({'ImageId': range(1, len(my_predictions)+1, 1), 'Label': my_predictions})
output_file

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [56]:
output_file.to_csv('pred4.csv', index=False)